

### Тема: Обработка ошибок, пагинация и фильтрация данных в REST API

1. **Цель урока**:
   - Научиться правильно обрабатывать и возвращать ошибки.
   - Реализовать пагинацию и фильтрацию данных.
   - Разобраться с использованием контекста запросов для контроля тайм-аутов.

2. **План урока**:
   1. **Введение в обработку ошибок**:
      - Обсуждение важности правильной обработки ошибок и возврата информативных ответов.
      - Примеры стандартных HTTP-кодов: `400 Bad Request`, `404 Not Found`, `500 Internal Server Error`, и т.д.

   2. **Реализация централизованного подхода к обработке ошибок**:
      - Добавим обработку ошибок в CRUD-функции, чтобы возвращать информативные сообщения.

      ```go
      func handleError(c *gin.Context, statusCode int, message string) {
          c.JSON(statusCode, gin.H{"error": message})
      }

      func getBookByID(c *gin.Context) {
          id := c.Param("id")
          var book Book
          if err := db.First(&book, id).Error; err != nil {
              handleError(c, http.StatusNotFound, "Book not found")
              return
          }
          c.JSON(http.StatusOK, book)
      }
      ```

   3. **Пагинация данных**:
      - Добавим пагинацию для маршрута `GET /books`, чтобы возвращать результаты по частям.
      - Пример реализации:

        ```go
        func getBooks(c *gin.Context) {
            var books []Book
            var total int64
            page := c.DefaultQuery("page", "1")
            limit := c.DefaultQuery("limit", "10")

            pageInt, _ := strconv.Atoi(page)
            limitInt, _ := strconv.Atoi(limit)

            offset := (pageInt - 1) * limitInt

            db.Limit(limitInt).Offset(offset).Find(&books).Count(&total)

            c.JSON(http.StatusOK, gin.H{
                "data":  books,
                "total": total,
                "page":  pageInt,
                "limit": limitInt,
            })
        }
        ```

   4. **Фильтрация данных**:
      - Реализуем фильтрацию книг по автору и названию. Например, через параметры запроса `author` и `title`.

        ```go
        func getBooks(c *gin.Context) {
            var books []Book
            var total int64
            page := c.DefaultQuery("page", "1")
            limit := c.DefaultQuery("limit", "10")
            author := c.Query("author")
            title := c.Query("title")

            pageInt, _ := strconv.Atoi(page)
            limitInt, _ := strconv.Atoi(limit)

            offset := (pageInt - 1) * limitInt
            query := db.Limit(limitInt).Offset(offset)

            if author != "" {
                query = query.Where("author ILIKE ?", "%"+author+"%")
            }
            if title != "" {
                query = query.Where("title ILIKE ?", "%"+title+"%")
            }

            query.Find(&books).Count(&total)

            c.JSON(http.StatusOK, gin.H{
                "data":  books,
                "total": total,
                "page":  pageInt,
                "limit": limitInt,
            })
        }
        ```

   5. **Использование контекста запроса для тайм-аутов**:
      - Применим контекст запроса для контроля тайм-аутов в обработке запросов. Это особенно полезно при длительных операциях.

        ```go
        func getBooksWithTimeout(c *gin.Context) {
            ctx, cancel := context.WithTimeout(c.Request.Context(), 2*time.Second)
            defer cancel()

            var books []Book
            if err := db.WithContext(ctx).Find(&books).Error; err != nil {
                handleError(c, http.StatusRequestTimeout, "Request timed out")
                return
            }

            c.JSON(http.StatusOK, books)
        }
        ```

3. **Практическая часть**:
   - Добавить пагинацию и фильтрацию в существующие маршруты.
   - Реализовать новые методы с использованием контекста для контроля тайм-аутов.

4. **Задание на дом**:
   - Добавить возможность сортировки данных по полям (например, `title`, `author`) с помощью параметра запроса `sort`.
   - Реализовать обработку ошибок для нестандартных ситуаций, например, попытка обновления несуществующего ресурса.
